In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
import pandas as pd
!pip install adversarial-robustness-toolbox
import torch.optim as optim
import numpy as np

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist



matplotlib.rcParams['figure.facecolor'] = '#ffffff'
import os
user = os.uname().nodename
import datetime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dataset = MNIST(root='data/', download=True, transform=ToTensor())

In [ ]:
val_size = 10000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(50000, 10000)

In [ ]:
batch_size=128
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for images, labels in train_loader:
    print('images.shape:', images.shape)
    inputs = images.reshape(-1, 784)
    print('inputs.shape:', inputs.shape)
    break

images.shape: torch.Size([128, 1, 28, 28])
inputs.shape: torch.Size([128, 784])


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [ ]:
input_size = inputs.shape[-1]
hidden1_size = 256
layer1 = nn.Linear(input_size, hidden1_size)
inputs.shape

torch.Size([128, 784])

In [ ]:
layer1_outputs = layer1(inputs)
print('layer1_outputs.shape:', layer1_outputs.shape)

layer1_outputs.shape: torch.Size([128, 256])


In [ ]:
layer1_outputs_direct = inputs @ layer1.weight.t() + layer1.bias
layer1_outputs_direct.shape

torch.Size([128, 256])

In [ ]:
torch.allclose(layer1_outputs, layer1_outputs_direct, 1e-3)

True

In [ ]:
relu1_outputs = F.relu(layer1_outputs)
print('min(layer1_outputs):', torch.min(layer1_outputs).item())
print('min(relu1_outputs):', torch.min(relu1_outputs).item())

min(layer1_outputs): -0.7752687335014343
min(relu1_outputs): 0.0


In [ ]:
hidden2_size = 32
layer2 = nn.Linear(hidden1_size, hidden2_size)

In [ ]:
layer2_outputs = layer2(relu1_outputs)
print(layer2_outputs.shape)

torch.Size([128, 32])


In [ ]:
output_size = 10
layer3 = nn.Linear(hidden2_size,output_size)

In [ ]:
relu2_outputs = F.relu(layer2_outputs)
print('min(layer2_outputs):', torch.min(layer2_outputs).item())
print('min(relu2_outputs):', torch.min(relu2_outputs).item())

min(layer2_outputs): -0.35356763005256653
min(relu2_outputs): 0.0


In [ ]:
layer3_outputs = layer3(relu2_outputs)
print(layer3_outputs.shape)

torch.Size([128, 10])


In [ ]:
inputs.shape

torch.Size([128, 784])

In [ ]:
F.cross_entropy(layer3_outputs, labels)
# Expanded version of layer2(F.relu(layer1(inputs)))
inner = F.relu(inputs @ layer1.weight.t() + layer1.bias)
middle = F.relu(inner @ layer2.weight.t() + layer2.bias)
outer = ((middle)) @ layer3.weight.t() + layer3.bias
outputs = outer

In [ ]:
torch.allclose(outputs, layer3_outputs, 1e-3)

True

In [ ]:
# Same as layer3(layer2(layer1(inputs)))
outputs3 = ((inputs @ layer1.weight.t() + layer1.bias) @ layer2.weight.t() + layer2.bias) @ layer3.weight.t() + layer3.bias

In [ ]:
# Create a single layer to replace the 3 linear layers
combined_layer = nn.Linear(input_size, output_size)

combined_layer.weight.data = layer2.weight @ layer1.weight
combined_layer.bias.data = layer1.bias @ layer2.weight.t() + layer2.bias

combined_layer.weight.data = layer3.weight @ combined_layer.weight.data
combined_layer.bias.data = combined_layer.bias.data @ layer3.weight.t() + layer3.bias

In [ ]:
# Same as combined_layer(inputs)
outputs4 = inputs @ combined_layer.weight.t() + combined_layer.bias

In [ ]:
torch.allclose(outputs3, outputs4, 1e-3)

True

In [ ]:
class MnistModel(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden1_size,hidden2_size,out_size):
        super().__init__()
        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden1_size)
        self.linear2 = nn.Linear(hidden1_size,hidden2_size)
        # output layer
        self.linear3 = nn.Linear(hidden2_size, out_size)
        
    def forward(self, xb):
        # Flatten the image tensors
        xb = xb.view(xb.size(0), -1)
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        out = F.relu(out) #this is what you removed
        # Get predictions using output layer
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
input_size = 784
hidden1_size = 256
hidden2_size = 32# you can change this
num_classes = 10

In [ ]:
model = MnistModel(input_size, hidden1_size=256,hidden2_size = 32, out_size=num_classes)

In [ ]:
#doubtful

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

In [ ]:
testno = 0
log_3layers = {}
log3layers = {}
num_epochs = 1
learning_rate = 0.01

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def accuracy(predictions):
    return (np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test))

In [ ]:
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)


In [ ]:
def trainandtest(n):
    for i in range(n):
        global testno
        testno+=1
        classifier.fit(x_train, y_train, batch_size=128, nb_epochs=num_epochs)
        benign_predictions = classifier.predict(x_test)
        benign_accuracy = accuracy(benign_predictions)*100
        intensity = 0.3
        attack = FastGradientMethod(estimator=classifier, eps=intensity)
        x_test_adv = attack.generate(x=x_test)
        
        
        adv_predictions = classifier.predict(x_test_adv)
        adv_accuracy = accuracy(adv_predictions) * 100
        e = datetime.datetime.now()
        log3layers[testno] = {'Test Number':testno,
                              'Learning Rate': learning_rate,
                              'Number of Epochs':num_epochs,
                              'Accuracy with Benign Test Set':benign_accuracy,
                              'Accuracy with Adversarial Test Set':adv_accuracy,
                              'EPS':intensity,
                             'Machine':user
                             }
        print("iteration ",i+1," over")
        


In [ ]:
trainandtest(100)

iteration  1  over
iteration  2  over
iteration  3  over
iteration  4  over
iteration  5  over
iteration  6  over
iteration  7  over
iteration  8  over
iteration  9  over
iteration  10  over
iteration  11  over
iteration  12  over
iteration  13  over
iteration  14  over
iteration  15  over
iteration  16  over
iteration  17  over
iteration  18  over
iteration  19  over
iteration  20  over
iteration  21  over
iteration  22  over
iteration  23  over
iteration  24  over
iteration  25  over
iteration  26  over
iteration  27  over
iteration  28  over
iteration  29  over
iteration  30  over
iteration  31  over
iteration  32  over
iteration  33  over
iteration  34  over
iteration  35  over
iteration  36  over
iteration  37  over
iteration  38  over
iteration  39  over
iteration  40  over
iteration  41  over
iteration  42  over
iteration  43  over
iteration  44  over
iteration  45  over
iteration  46  over
iteration  47  over
iteration  48  over
iteration  49  over
iteration  50  over
iteration

In [ ]:
df = pd.DataFrame(data = log3layers)
df.T

NameError: ignored